# Un signal peut en cacher un autre...
## Déconvolution de spectres UV-Vis pour le stockage de l'Energie  avec des Batteries Redox Flow au Vanadium

Les batteries *Redox Flow* au Vanadium permettent de stocker l'énergie sous frome chimique et de la restituer à la demande. Cette technologie est envisagée pour stocker l'énergie produite par les sources d'énergies renouvellables intermittantes (solaire, éolien...).

Une batterie Redox Flow au Vanadium est schématisée dans l'image ci-dessous à gauche.

<img src="im1.png"/>

La batterie opère avec des solutions acides de Vanadium (dissous) à différents degrés d'oxydation (II, III, IV et V). Les solutions présentent des couleurs différentes en fonction de leurs teneurs en chacun des degrés d'oxydation (voir image ci-dessus à droite).

Lorsqu'on étudie le fonctionnement d'une telle batterie, on peut alors utiliser la spectroscopie UV-Vis pour déterminer la composition des solutions pour un état de charge (*State of Charge*) donné.

Une complication, assez commune, est que le Vanadium peut exister simultanément sous différentes formes, i.e. à différents degrés d'oxydation, et donc le spectre UV-Vis résultant est composé de la superposition des spectres de chaque forme de Vanadium. 

### Objectif du projet:
A partir du spectre UV-Vis d'une solution de batterie Redox Flow Vanadium (fichier data_uv.dat), voir ci-dessous, remonter à la compostion en V(II), V(III), V(IV) et V(V).
<img src="im_uv.png" width = 400 />


### Méthode : Traitement du signal du spectre UV-Vis fourni
La principales étapes seront :

+ la soustraction de la ligne de base
+ la *déconvolution* du signal, en faisant l'hypothèse que le signal est la somme de plusieurs gaussiennes, puis ajustement par les moindres carrés (curve_fit) pour trouver la position, l'amplitude et la largeur de chaque gaussienne
+ l'utilisation des calibrations (fournies)


## Etape 1 : détermination de la ligne de base

**Description** : il est courant que le signal à traiter soit décallé vers le haut et semble "porté par une courbe" que l'on appelle **ligne de base** (même si un blanc a été effectué au préalable), voir un exemple ci-dessous. Le signal entre les pics ne retombe pas à 0 et la hauteur des minimums est variable le long de l'axe des $x$.

<img src="im6.png" width = 400 />

Il existe différentes méthodes pour déterminer cette ligne de base afin de la retrancher au signal pour le corriger . **Il faut faire cette correction afin de pouvoir traiter le spectre et en tirer les informations recherchées.**


Ici, on se propose de commencer en utilisant une ligne de base linéaire, puis de voir ce que ça change si on utilise une méthode plus évoluée basée sur les moindres carrés assymétriques ALS (*Asymmetric Least Squares*). 

**Instructions :**

1) charger les données du spectre contenues dans le fichier data_uv.dat (première colonne = longueur d'onde en nm, seconde colonne = absorbance) et tracer le spectre


2) pour obtenir la ligne de base linéaire du spectre, écrire un code qui permet d'obtenir l'équation de la droite qui passe au plus près des 3 minimums du spectre, vers 300, 500 et 1000 nm. On utilisera curve_fit et on déterminera "manuellement" les coordonnées des 3 minimums

3) tracer la ligne de base, le spectre et le spectre corrigé sur la même figure (avec légende)


In [1]:
# 

## Etape 2 : ajustement par somme de gaussiennes

On va ajuster les pics du spectre corrigé par des fonctions gaussiennes qui ont pour formule :
\begin{equation}
g_{\sigma, x_0}(x) = \frac{1}{\sigma \sqrt{2 \pi}} e^{-\frac{1}{2}\left(\frac{x-x_0}{\sigma}\right)^2},
\end{equation}
où $\sigma$ correspond à la "largeur" (ou dispersion) du pic et $x_0$ à sa position en $x$.

Notre spectre contient visiblement 3 pics, donc on va ajuster le signal corrigé par une fonction $G(x)$ qui sera la somme de 3 fonctions guaussiennes définie par :
\begin{equation}
G(x) = \sum_{i=1}^{3} A_i g_{\sigma_i, {x_0}_i}(x),
\end{equation}
où $A_i$, $\sigma_i$ et ${x_0}_i$,  sont respectivement l'amplitude, la largeur et la position de chaque gaussienne. Ces paramètres vont être déterminés par ajustement.

**Instructions :**

1) Ecrire une fonction python qui permet de calculer la valeur d'une fonction gaussienne pour un $x$ donné, $g_{\sigma, x_0}(x)$, en fonction des paramètres $\sigma$  et $x_0$. Tracer une telle fonction (test)

2) Ecrire une fonction python qui permet de calculer la somme de 3 fonctions gaussiennes pour un $x$ donné, $G(x)$, en fonction des paramètres $A_1$, $\sigma_1$, $x_1$, $A_2$, $\sigma_2$, $x_2$, $A_3$, $\sigma_3$, $x_3$.

3) Ajuster le spectre corrigé avec la fonction à 3 gaussiennes; attention il faudra initialiser la recherche des paramètres et donc utiliser l'argument p0 de curve_fit (voir doc de scipy), utiliser des valeurs proches de celles attendues notamment les positions des pics

4) Tracer le signal original et le signal ajusté, ainsi que chaque fonction gaussienne, sur une même figure et afficher les valeurs des postions $x_0$ et des amplitudes

In [2]:
#

## Etape 3 : utilisation des données de calibration

A partir des amplitudes et des positions de pics détermineés, on a va remonter aux concentrations de chaque degré d'oxydation du vanadium. 
Pour cela, nous avons besoin de données de calibration pour chaque degré d'oxydation, les voici ci-dessous (issues de la El Hage 2020 [1]).


### Spectres UV-Vis des différentes formes du Vanadium
<img src="im2.png" width = 600/>

### Vanadium II
<img src="im5.png" width = 600/ />

### Vanadium III 
<img src="im3.png" width = 600/ />

### Vanadium IV
<img src="im4.png" width = 600/ />


**Instructions :**
1) Déterminer approximativement les pentes des courbes d'étallonage, $y=ax$, pour le V(II), le V(III) et le V(IV)

2) En déduire les concentrations de chaque forme de Vanadium dans l'échantillon analysé


**[1] R. El Hage, Etude et optimisation d'une batterie à circulation tout vanadium, 2020, Thèse de doctorat, Université Paul Sabatier-Toulouse III.**

In [3]:
#

## Amélioration de la méthode
Recommencer le traitement en utilisant cette fois-ci une méthode plus évoluée pour l'obtention de la ligne de base : moindres carrés assymétriques.

En gros, il s'agit d'une méthode d'ajustement où le poids des minimums est très grand devant celui des autres points du signal. La ligne de base ne sera pas linéaire et "collera" plus aux minimums locaux, on s'attend donc à un meilleur résultat. Vous n'avez pas besoin de connaître dans le détail cette méthode, on vous propose juste de l'utiliser. 

Le script de la méthode ALS [2] est fourni ci-dessous sous forme d'une fonction python (baseline_als). Cette fonction est à utiliser tel quel. Pour qu'elle fonctionne, elle a besoin des modules sparse et spsolve de scipy, penser à les importer (voir ci-dessous).

Description de la fonction baseline_als :
+ argument y : le signal pour lequel on veut obtenir la ligne de base
+ arguments lam et p : les paramètres de la méthode ALS, lam est à prendre dans la gamme 10$^4$ - 10$^9$, et p dans la gamme 0.1 - 0.0001
+ argument niter = nombre d'itérations, le laisser = à 10
+ variable renvoyée : les valeurs de la ligne de base (tableau python)

Il faut rechercher les "bonnes" valeurs de lam et p en les faisant varier et en observant comment est affecté la ligne de base calculée.


[2] https://www.originlab.com/doc/en/Origin-Help/PeakAnalyzer-ALSBaseline, 
Oller-Moreno, S., Pardo, A., Jiménez-Soto, J. M., Samitier, J., & Marco, S. (2014, February). Adaptive Asymmetric Least Squares baseline estimation for analytical instruments. In 2014 IEEE 11th International Multi-Conference on Systems, Signals & Devices (SSD14) (pp. 1-5). IEEE.

In [4]:
#